In [30]:
import scipy.stats as stats
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
def simulate_gbm_time_series(r,q,sigma,T,num_divisons,spot):
    delta_t=T*1.0/num_divisons
    S=spot
    time_series=[]
    time_series.append(S)
    nu=(r-q-(sigma**2/2))
    for i in range(num_divisons):
        z=np.random.standard_normal(1)
        S=S*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*z)
        time_series.append(S)
    return time_series

def simulate_gbm_time_series_antithetic(r,q,sigma,T,num_divisons,spot):
    delta_t=T*1.0/num_divisons
    S=spot
    Sa=spot
    time_series=[]
    time_series_a=[]
    time_series.append(S)
    time_series_a.append(S)
    nu=(r-q-(sigma**2/2))
    for i in range(num_divisons):
        z=np.random.standard_normal(1)
        S=S*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*z)
        Sa=Sa*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*-z)
        time_series.append(S)
        time_series_a.append(Sa)
    return [time_series,time_series_a]

def calculate_price_ci_asian_call(r,q,sigma,T,spot,K,n_simulation_paths,alpha,resampling_rate):
    Y=[]
    price=0
    discouting_factor=math.exp(-r*T) 
    
    # sampling payoffs
    for i in range(n_simulation_paths):
        time_series=simulate_gbm_time_series(r,q,sigma,T,resampling_rate*20,spot)[::20]
        Y.append(discouting_factor*max(np.mean(time_series)-K,0))

    
    price=np.mean(Y)
    sample_std=np.std(Y)
    
    print("Standard Monte-Carlo variance: "+str(np.var(Y)/n_simulation_paths))

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,discouting_factor*lower_bound_clt,upper_bound_clt)



r=0.04
q=0.015
T=1
sigma=0.30
spot=1000
K=1100
m=15
n_simulation_paths=50000
alpha=0.01


print("----Standard Monte-Carlo CI------")
(price,lb,ub)=calculate_price_ci_asian_call(r,q,sigma,T,spot,K,2*n_simulation_paths,alpha,m)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub))





----Standard Monte-Carlo CI------
Standard Monte-Carlo variance: 0.06949247440579331
Price: 35.64156586433876 LB: 33.59163972303649 UB:36.32059118215213


In [32]:
def calculate_price_ci_asian_call_w_antithetic(r,q,sigma,T,spot,K,n_simulation_paths,alpha,resampling_rate):
    Y=[]
    price=0
    discouting_factor=math.exp(-r*T) 
    
    # sampling payoffs
    for i in range(n_simulation_paths):
        [time_series,time_series_a]=simulate_gbm_time_series_antithetic(r,q,sigma,T,resampling_rate*20,spot)
        time_series=time_series[::20]
        time_series_a=time_series_a[::20]
        C1=discouting_factor*max(np.mean(time_series)-K,0)
        C2=discouting_factor*max(np.mean(time_series_a)-K,0)
        Y.append((C1+C2)/2)

    
    price=np.mean(Y)
    sample_std=np.std(Y)
    
    print("w/ Antithetic Monte-Carlo variance: "+str(np.var(Y)/n_simulation_paths))

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,lower_bound_clt,upper_bound_clt)


print("----Antithetic CI------")
(price,lb,ub)=calculate_price_ci_asian_call_w_antithetic(r,q,sigma,T,spot,K,n_simulation_paths,alpha,m)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub))        


----Antithetic CI------
w/ Antithetic Monte-Carlo variance: 0.05668528359326085
Price: 35.50864987073173 LB: 34.89537913488919 UB:36.12192060657427
